# Data Preprocessor for the Controller
The controller receives the z produced by the encoder (from the VAE) and the z' produced by the RNN's prediction to determine an action.
The below processor takes a collection of image data and the action performed for preprocessing. It finds z and z' by putting the image through the encoder and then putting z through the RNN.
In the form I left it in, we did not have a good dataset of images and corresponding images. Right now, it just sets every action to 22 (no action).

## Loading stuff into Kernal

imports

`import sys`: added so that cv2 gets installed in kernal

`import sys`
`!{sys.executable} -m pip install opencv-python`
commented the above code, it started working, 
idk why if code not working try uncommenting the above

In [ ]:
import tensorflow.compat.v1.keras.backend as K
import tensorflow as tf
tf.compat.v1.disable_eager_execution()
from tensorflow import keras
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, Input, Flatten, Dense, Lambda, Reshape, MaxPooling2D, Dropout, LSTM
from tensorflow.keras.models import Model
from tensorflow.keras.models import save_model
from tensorflow.keras.datasets import mnist
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam
import numpy as np
import matplotlib.pyplot as plt
import os
import pickle
import joblib
# added so that cv2 gets installed in kernal
# import sys
# !{sys.executable} -m pip install opencv-python
# commented the above code, it started working, idk why
# if code not working try uncommenting the above
import cv2
import random
import glob

# Data (preprocessed from Data Processing Script)

In [ ]:
train_data = joblib.load("images/train_data.z")
print(train_data.shape[2])

img_width = train_data.shape[1]
img_height = train_data.shape[2]
num_channels = 1
x_train = train_data.reshape(train_data.shape[0], img_height, img_width, num_channels)

input_shape = (img_height, img_width, num_channels)
print(input_shape)



### load the vae (have to make the architecture again, make sure the code below

matches the code in the Data Prepper/VAE Trainer)


# Encoder

`x`: The Encoder Model

`latent_dim`: changing this will make the model exponentially larger or smaller

`conv_shape`: Shape of conv to be provided to decoder

`z_mu` and `z_sigma`: Two outputs, for latent mean and log variance (std. dev.)
Use these to sample random variables in latent space to which inputs

`z_mu`: Mean values of encoded input
`z_sigma`: Std dev. (variance) of encoded input

`sample_z()` Function: REPARAMETERIZATION TRICK
- Define sampling function to sample from the distribution
- Reparameterize sample based on the process defined by Gunderson and Huang
- into the shape of: mu + sigma squared x eps
- This is to allow gradient descent to allow for gradient estimation accurately. 

`z`: sample vector from the latent distribution `z` is the labda custom layer we are adding for gradient descent calculations using mu and variance (sigma)

`encoder`: Z (lambda layer) will be the last layer in the encoder. Define and summarize encoder model.


In [ ]:
latent_dim = 2048

input_img = Input(shape=input_shape, name='encoder_input')
x = Conv2D(64, 3, padding='same', activation='relu')(input_img)
x = MaxPooling2D((2,2), padding='same')(x)
x = Dropout(0.2)(x)
x = Conv2D(128, 3, padding='same', activation='relu')(x)
x = MaxPooling2D((2,2), padding='same')(x)
x = Dropout(0.2)(x)
x = Conv2D(64, 3, padding='same', activation='relu')(x)
x = MaxPooling2D((2,2), padding='same')(x)
x = Dropout(0.2)(x)
x = Conv2D(32, 3, padding='same', activation='relu')(x)

conv_shape = K.int_shape(x)
x = Flatten()(x)
x = Dense(latent_dim*2, activation='relu')(x)

z_mu = Dense(latent_dim, name='latent_mu')(x)
z_sigma = Dense(latent_dim, name='latent_sigma')(x)

def sample_z(args):
    z_mu, z_sigma = args
    eps = K.random_normal(shape=(K.shape(z_mu)[0], K.int_shape(z_mu)[1]))
    return z_mu + K.exp(z_sigma / 2) * eps

z = Lambda(sample_z, output_shape=(latent_dim,), name='z')([z_mu, z_sigma])

encoder = Model(input_img, [z_mu, z_sigma, z], name='encoder')
print(encoder.summary())



# Decoder

decoder takes the latent vector as input

`x`: Need to start with a shape that can be remapped to original image shape as we want our final utput to be same shape original input. So, add dense layer with dimensions that can be reshaped to desired output shape

`x = Reshape()`: reshape to the shape of last conv. layer in the encoder, so we can upscale (conv2D transpose) back to original shape use Conv2DTranspose to reverse the conv layers defined in the encoder
(`Conv2DTranspose()` Layers)

Last `Conv2DTranspose()` Layer: Using sigmoid activation  (Can add more conv2DTranspose layers, if desired.)

`z_decoded`: apply the decoder to the latent sample 

In [ ]:
decoder_input = Input(shape=(latent_dim, ), name='decoder_input')

x = Dense(conv_shape[1]*conv_shape[2]*conv_shape[3], activation='relu')(decoder_input)
x = Reshape((conv_shape[1], conv_shape[2], conv_shape[3]))(x)
x = Conv2DTranspose(32, 3, padding='same', activation='relu', strides=(2,2))(x)
x = Conv2DTranspose(32, 3, padding='same', activation='relu')(x)
x = Conv2DTranspose(64, 3, padding='same', activation='relu', strides=(2,2))(x)
x = Conv2DTranspose(64, 3, padding='same', activation='relu')(x)
x = Conv2DTranspose(64, 3, padding='same', activation='relu', strides=(2,2))(x)

x = Conv2DTranspose(num_channels, 3, padding='same', activation='sigmoid', name='decoder_output')(x)

decoder = Model(decoder_input, x, name='decoder')
decoder.summary()

z_decoded = decoder(z)

# Loss Function

- `recon_loss`: Reconstruction loss (as we used sigmoid activation we can use binarycrossentropy)
- `kl_loss`: KL divergence
- `call()`: add custom loss to the class

In [ ]:
def CustomLayer(keras.layers.Layer):
    def vae_loss(self, x, z_decoded):
        x = K.flatten(x)
        z_decoded = K.flatten(z_decoded)
        
        recon_loss = keras.metrics.binary_crossentropy()